In [1]:
# Load packages
import pickle

from FDApy.preprocessing.dim_reduction.fpca import UFPCA
from FDApy.representation.functional_data import DenseFunctionalData
from FDApy.clustering.fcubt import Node, FCUBT

from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics import adjusted_rand_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle

In [2]:
# Load data
with open('./data/scenario_1.pkl', 'rb') as f:
    data_fd = pickle.load(f)
with open('./data/labels.pkl', 'rb') as f:
    labels = pickle.load(f)

In [3]:
# Split data into train/test set
values, new_labels = shuffle(data_fd.values, labels, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(values, new_labels, test_size=0.33, random_state=42)

data_train = DenseFunctionalData(data_fd.argvals, X_train)
data_test = DenseFunctionalData(data_fd.argvals, X_test)

## Perform UFPCA

In [4]:
# Do UFPCA on the data
fpca = UFPCA(n_components=0.99)
fpca.fit(data_train, method='GAM')
    
# Compute scores
train_proj = fpca.transform(data_train, method='NumInt')
test_proj = fpca.transform(data_test, method='NumInt')

## Classification using GaussianProcessClassifier

In [5]:
# Fit GPC
gp = GaussianProcessClassifier(1.0 * RBF(1.0))
gp.fit(train_proj, y_train)

GaussianProcessClassifier(kernel=1**2 * RBF(length_scale=1))

In [6]:
# Prediction for GPC
pred_gpc = gp.predict(test_proj)

## CART classification

In [7]:
# Fit CART
clf = RandomForestClassifier()
clf.fit(train_proj, y_train)

RandomForestClassifier()

In [8]:
# Prediction for CART
pred_cart = clf.predict(test_proj)

## fCUBT clustering

In [9]:
# Initialization of the tree
root_node = Node(data_test, is_root=True)
fcubt = FCUBT(root_node=root_node)

In [10]:
# Grow and join the tree
fcubt.grow(n_components=0.95)
fcubt.join(n_components=0.95)

## Results

In [11]:
print(f'Adjusted Rand Index for FCUBT: {adjusted_rand_score(y_test, fcubt.labels_join)}')
print(f'Adjusted Rand Index for GPC: {adjusted_rand_score(y_test, pred_gpc)}')
print(f'Adjusted Rand Index for CART: {adjusted_rand_score(y_test, pred_cart)}')

Adjusted Rand Index for FCUBT: 0.8655359514871155
Adjusted Rand Index for GPC: 0.7882195738695846
Adjusted Rand Index for CART: 0.8249880264941892
